In [1]:
import pandas as pd
import sklearn
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# col_name=['adult','backdrop_path','movie_id','original_language','original_title','overview','popularity','poster_path','release_date','title','video','vote_average','vote_count','genres','keywords','cast','crew']
df=pd.read_csv('./dataset/CleanedTMDB1000.csv')
df

,Unnamed:0,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,keywords,cast,crew
0,0,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Crime']",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",124.718,https://image.tmdb.org/t/p/original//3bhkrj58V...,1972-03-14,The Godfather,False,8.7,18148,"[{'id': 131, 'name': 'italy'}, {'id': 697, 'na...","[{'adult': False, 'gender': 2, 'id': 3084, 'kn...","[{'adult': False, 'gender': 2, 'id': 154, 'kno..."
1,1,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Crime']",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,92.927,https://image.tmdb.org/t/p/original//lyQBXzOQS...,1994-09-23,The Shawshank Redemption,False,8.7,24030,"[{'id': 378, 'name': 'prison'}, {'id': 417, 'n...","[{'adult': False, 'gender': 2, 'id': 504, 'kno...","[{'adult': False, 'gender': 2, 'id': 153, 'kno..."
2,2,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Crime']",240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,61.583,https://image.tmdb.org/t/p/original//bMadFzhjy...,1974-12-20,The Godfather Part II,False,8.6,10955,"[{'id': 131, 'name': 'italy'}, {'id': 700, 'na...","[{'adult': False, 'gender': 2, 'id': 1158, 'kn...","[{'adult': False, 'gender': 2, 'id': 154, 'kno..."
3,3,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'History', 'War']",424,en,Schindler's List,The true story of how businessman Oskar Schind...,48.961,https://image.tmdb.org/t/p/original//sF1U4EUQS...,1993-12-15,Schindler's List,False,8.6,14203,"[{'id': 818, 'name': 'based on novel or book'}...","[{'adult': False, 'gender': 2, 'id': 3896, 'kn...","[{'adult': False, 'gender': 2, 'id': 491, 'kno..."
4,4,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Comedy', 'Drama', 'Romance']",19404,hi,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...",28.639,https://image.tmdb.org/t/p/original//ktejodbcd...,1995-10-19,Dilwale Dulhania Le Jayenge,False,8.6,4157,"[{'id': 2783, 'name': ""family's daily life""}, ...","[{'adult': False, 'gender': 2, 'id': 35742, 'k...","[{'adult': False, 'gender': 1, 'id': 8311, 'kn..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Horror', 'Thriller']",300467,en,Dark Was the Night,An evil is unleashed in a small town when a lo...,8.134,https://image.tmdb.org/t/p/original//xSC7sWXba...,2014-10-17,Dark Was the Night,False,5.6,223,"[{'id': 447, 'name': 'post traumatic stress di...","[{'adult': False, 'gender': 2, 'id': 79072, 'k...","[{'adult': False, 'gender': 2, 'id': 59657, 'k..."
9996,9996,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Action', 'Crime', 'Thriller']",106747,en,Machete Kills,Ex-Federale agent Machete is recruited by the ...,14.935,https://image.tmdb.org/t/p/original//3i4UgSZmA...,2013-09-12,Machete Kills,False,5.6,1717,"[{'id': 534, 'name': 'mexico'}, {'id': 833, 'n...","[{'adult': False, 'gender': 2, 'id': 11160, 'k...","[{'adult': False, 'gender': 2, 'id': 2294, 'kn..."
9997,9997,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Adventure', 'Comedy', 'Family', 'Romance']",10603,en,George of the Jungle,"Deep in the African jungle, a baby named Georg...",16.061,https://image.tmdb.org/t/p/original//lWp8hUqE4...,1997-07-15,George of the Jungle,False,5.6,1875,"[{'id': 409, 'name': 'africa'}, {'id': 582, 'n...","[{'adult': False, 'gender': 2, 'id': 18269, 'k...","[{'adult': False, 'gender': 2, 'id': 373, 'kno..."
9998,9998,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Horror', 'Thriller']",692969,no,Kadaver,"In the aftermath of a nuclear disaster, a star...",13.666,https://image.tmdb.org/t/p/original//bo94ZQ0Sl...,2020-10-22,Cadaver,False,5.6,553,"[{'id': 612, 'name': 'hotel'}, {'id': 2546, 'n

In [3]:
df.columns

Index(['Unnamed:0', 'adult', 'backdrop_path', 'genre_ids', 'id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'release_date', 'title', 'video', 'vote_average',
       'vote_count', 'keywords', 'cast', 'crew'],
      dtype='object')

In [4]:
df.describe()

,Unnamed:0,id,popularity,vote_average,vote_count
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000
mean,4999.50000,1.826732e+05,23.429649,6.762800,1656.985500
std,2886.89568,2.386496e+05,69.101463,0.647175,2858.642357
min,0.00000,2.000000e+00,0.600000,5.600000,200.000000
25%,2499.75000,1.031375e+04,9.971000,6.200000,320.000000
50%,4999.50000,3.503850e+04,13.595500,6.700000,611.000000
75%,7499.25000,3.439608e+05,21.003750,7.200000,1572.250000
max,9999.00000,1.102776e+06,3980.086000,8.700000,33908.000000


In [5]:
import ast

In [6]:
def convert(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            l.append(str(i['name']))
            counter+=1
        else:
            break
        return l

In [7]:
df['cast']=df['cast'].apply(convert)
df

,Unnamed:0,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,keywords,cast,crew
0,0,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Crime']",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",124.718,https://image.tmdb.org/t/p/original//3bhkrj58V...,1972-03-14,The Godfather,False,8.7,18148,"[{'id': 131, 'name': 'italy'}, {'id': 697, 'na...",[Marlon Brando],"[{'adult': False, 'gender': 2, 'id': 154, 'kno..."
1,1,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Crime']",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,92.927,https://image.tmdb.org/t/p/original//lyQBXzOQS...,1994-09-23,The Shawshank Redemption,False,8.7,24030,"[{'id': 378, 'name': 'prison'}, {'id': 417, 'n...",[Tim Robbins],"[{'adult': False, 'gender': 2, 'id': 153, 'kno..."
2,2,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Crime']",240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,61.583,https://image.tmdb.org/t/p/original//bMadFzhjy...,1974-12-20,The Godfather Part II,False,8.6,10955,"[{'id': 131, 'name': 'italy'}, {'id': 700, 'na...",[Al Pacino],"[{'adult': False, 'gender': 2, 'id': 154, 'kno..."
3,3,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'History', 'War']",424,en,Schindler's List,The true story of how businessman Oskar Schind...,48.961,https://image.tmdb.org/t/p/original//sF1U4EUQS...,1993-12-15,Schindler's List,False,8.6,14203,"[{'id': 818, 'name': 'based on novel or book'}...",[Liam Neeson],"[{'adult': False, 'gender': 2, 'id': 491, 'kno..."
4,4,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Comedy', 'Drama', 'Romance']",19404,hi,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...",28.639,https://image.tmdb.org/t/p/original//ktejodbcd...,1995-10-19,Dilwale Dulhania Le Jayenge,False,8.6,4157,"[{'id': 2783, 'name': ""family's daily life""}, ...",[Shah Rukh Khan],"[{'adult': False, 'gender': 1, 'id': 8311, 'kn..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Horror', 'Thriller']",300467,en,Dark Was the Night,An evil is unleashed in a small town when a lo...,8.134,https://image.tmdb.org/t/p/original//xSC7sWXba...,2014-10-17,Dark Was the Night,False,5.6,223,"[{'id': 447, 'name': 'post traumatic stress di...",[Kevin Durand],"[{'adult': False, 'gender': 2, 'id': 59657, 'k..."
9996,9996,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Action', 'Crime', 'Thriller']",106747,en,Machete Kills,Ex-Federale agent Machete is recruited by the ...,14.935,https://image.tmdb.org/t/p/original//3i4UgSZmA...,2013-09-12,Machete Kills,False,5.6,1717,"[{'id': 534, 'name': 'mexico'}, {'id': 833, 'n...",[Danny Trejo],"[{'adult': False, 'gender': 2, 'id': 2294, 'kn..."
9997,9997,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Adventure', 'Comedy', 'Family', 'Romance']",10603,en,George of the Jungle,"Deep in the African jungle, a baby named Georg...",16.061,https://image.tmdb.org/t/p/original//lWp8hUqE4...,1997-07-15,George of the Jungle,False,5.6,1875,"[{'id': 409, 'name': 'africa'}, {'id': 582, 'n...",[Brendan Fraser],"[{'adult': False, 'gender': 2, 'id': 373, 'kno..."
9998,9998,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Horror', 'Thriller']",692969,no,Kadaver,"In the aftermath of a nuclear disaster, a star...",13.666,https://image.tmdb.org/t/p/original//bo94ZQ0Sl...,2020-10-22,Cadaver,False,5.6,553,"[{'id': 612, 'name': 'hotel'}, {'id': 2546, 'n...",[Gitte Witt],"[{'adult': False, 'gender': 2, 'id': 20308, 'k..."


In [8]:
def director(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(str(i['name']))
    return l

In [9]:
df['crew']=df['crew'].apply(director)
df

,Unnamed:0,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,keywords,cast,crew
0,0,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Crime']",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",124.718,https://image.tmdb.org/t/p/original//3bhkrj58V...,1972-03-14,The Godfather,False,8.7,18148,"[{'id': 131, 'name': 'italy'}, {'id': 697, 'na...",[Marlon Brando],[Francis Ford Coppola]
1,1,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Crime']",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,92.927,https://image.tmdb.org/t/p/original//lyQBXzOQS...,1994-09-23,The Shawshank Redemption,False,8.7,24030,"[{'id': 378, 'name': 'prison'}, {'id': 417, 'n...",[Tim Robbins],[Frank Darabont]
2,2,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Crime']",240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,61.583,https://image.tmdb.org/t/p/original//bMadFzhjy...,1974-12-20,The Godfather Part II,False,8.6,10955,"[{'id': 131, 'name': 'italy'}, {'id': 700, 'na...",[Al Pacino],[Francis Ford Coppola]
3,3,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'History', 'War']",424,en,Schindler's List,The true story of how businessman Oskar Schind...,48.961,https://image.tmdb.org/t/p/original//sF1U4EUQS...,1993-12-15,Schindler's List,False,8.6,14203,"[{'id': 818, 'name': 'based on novel or book'}...",[Liam Neeson],[Steven Spielberg]
4,4,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Comedy', 'Drama', 'Romance']",19404,hi,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...",28.639,https://image.tmdb.org/t/p/original//ktejodbcd...,1995-10-19,Dilwale Dulhania Le Jayenge,False,8.6,4157,"[{'id': 2783, 'name': ""family's daily life""}, ...",[Shah Rukh Khan],[Aditya Chopra]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Horror', 'Thriller']",300467,en,Dark Was the Night,An evil is unleashed in a small town when a lo...,8.134,https://image.tmdb.org/t/p/original//xSC7sWXba...,2014-10-17,Dark Was the Night,False,5.6,223,"[{'id': 447, 'name': 'post traumatic stress di...",[Kevin Durand],[Jack Heller]
9996,9996,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Action', 'Crime', 'Thriller']",106747,en,Machete Kills,Ex-Federale agent Machete is recruited by the ...,14.935,https://image.tmdb.org/t/p/original//3i4UgSZmA...,2013-09-12,Machete Kills,False,5.6,1717,"[{'id': 534, 'name': 'mexico'}, {'id': 833, 'n...",[Danny Trejo],[Robert Rodriguez]
9997,9997,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Adventure', 'Comedy', 'Family', 'Romance']",10603,en,George of the Jungle,"Deep in the African jungle, a baby named Georg...",16.061,https://image.tmdb.org/t/p/original//lWp8hUqE4...,1997-07-15,George of the Jungle,False,5.6,1875,"[{'id': 409, 'name': 'africa'}, {'id': 582, 'n...",[Brendan Fraser],[Sam Weisman]
9998,9998,False,https://image.tmdb.org/t/p/original//xOA4MbyJZ...,"['Drama', 'Horror', 'Thriller']",692969,no,Kadaver,"In the aftermath of a nuclear disaster, a star...",13.666,https://image.tmdb.org/t/p/original//bo94ZQ0Sl...,2020-10-22,Cadaver,False,5.6,553,"[{'id': 612, 'name': 'hotel'}, {'id': 2546, 'n...",[Gitte Witt],[Jarand Herdal]


# preprocessing the data

In [10]:
movies=df[['genre_ids', 'title','overview','cast','crew','id']]
movies

,genre_ids,title,overview,cast,crew,id
0,"['Drama', 'Crime']",The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",[Marlon Brando],[Francis Ford Coppola],238
1,"['Drama', 'Crime']",The Shawshank Redemption,Framed in the 1940s for the double murder of h...,[Tim Robbins],[Frank Darabont],278
2,"['Drama', 'Crime']",The Godfather Part II,In the continuing saga of the Corleone crime f...,[Al Pacino],[Francis Ford Coppola],240
3,"['Drama', 'History', 'War']",Schindler's List,The true story of how businessman Oskar Schind...,[Liam Neeson],[Steven Spielberg],424
4,"['Comedy', 'Drama', 'Romance']",Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",[Shah Rukh Khan],[Aditya Chopra],19404
...,...,...,...,...,...,...
9995,"['Horror', 'Thriller']",Dark Was the Night,An evil is unleashed in a small town when a lo...,[Kevin Durand],[Jack Heller],300467
9996,"['Action', 'Crime', 'Thriller']",Machete Kills,Ex-Federale agent Machete is recruited by the ...,[Danny Trejo],[Robert Rodriguez],106747
9997,"['Adventure', 'Comedy', 'Family', 'Romance']",George of the Jungle,"Deep in the African jungle, a baby named Georg...",[Brendan Fraser],[Sam Weisman],10603
9998,"['Drama', 'Horror', 'Thriller']",Cadaver,"In the aftermath of a nuclear disaster, a star...",[Gitte Witt],[Jarand Herdal],692969


In [11]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   genre_ids  10000 non-null  object
 1   title      10000 non-null  object
 2   overview   9998 non-null   object
 3   cast       9982 non-null   object
 4   crew       10000 non-null  object
 5   id         10000 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 468.9+ KB


In [12]:
movies.isnull().sum()

genre_ids     0
title         0
overview      2
cast         18
crew          0
id            0
dtype: int64

In [13]:
#converting lists to string in all the fields
movies['genre_ids']=movies['genre_ids'].apply(lambda x: ''.join([char for char in x if char not in string.punctuation]).split())
movies['overview']=movies['overview'].apply(lambda x:str(x))#to resolve 'Float' object is not iterable error, we convert it into string
movies['overview']=movies['overview'].apply(lambda x:''.join([char for char in x if char not in string.punctuation]).split())#converting to string results in [ ' , " to be considered as well so removing these punctuations
# movies['tag']=movies['genre_ids']+movies['overview']+movies['original_language']+movies['cast']+movies['crew']
movies

C:\Users\Kashyap Ghimire\AppData\Local\Temp\ipykernel_14000\1989286070.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genre_ids']=movies['genre_ids'].apply(lambda x: ''.join([char for char in x if char not in string.punctuation]).split())
C:\Users\Kashyap Ghimire\AppData\Local\Temp\ipykernel_14000\1989286070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['overview']=movies['overview'].apply(lambda x:str(x))#to resolve 'Float' object is not iterable error, we convert it into string
C

,genre_ids,title,overview,cast,crew,id
0,"[Drama, Crime]",The Godfather,"[Spanning, the, years, 1945, to, 1955, a, chro...",[Marlon Brando],[Francis Ford Coppola],238
1,"[Drama, Crime]",The Shawshank Redemption,"[Framed, in, the, 1940s, for, the, double, mur...",[Tim Robbins],[Frank Darabont],278
2,"[Drama, Crime]",The Godfather Part II,"[In, the, continuing, saga, of, the, Corleone,...",[Al Pacino],[Francis Ford Coppola],240
3,"[Drama, History, War]",Schindler's List,"[The, true, story, of, how, businessman, Oskar...",[Liam Neeson],[Steven Spielberg],424
4,"[Comedy, Drama, Romance]",Dilwale Dulhania Le Jayenge,"[Raj, is, a, rich, carefree, happygolucky, sec...",[Shah Rukh Khan],[Aditya Chopra],19404
...,...,...,...,...,...,...
9995,"[Horror, Thriller]",Dark Was the Night,"[An, evil, is, unleashed, in, a, small, town, ...",[Kevin Durand],[Jack Heller],300467
9996,"[Action, Crime, Thriller]",Machete Kills,"[ExFederale, agent, Machete, is, recruited, by...",[Danny Trejo],[Robert Rodriguez],106747
9997,"[Adventure, Comedy, Family, Romance]",George of the Jungle,"[Deep, in, the, African, jungle, a, baby, name...",[Brendan Fraser],[Sam Weisman],10603
9998,"[Drama, Horror, Thriller]",Cadaver,"[In, the, aftermath, of, a, nuclear, disaster,...",[Gitte Witt],[Jarand Herdal],692969


# creating tags field in dataset

In [14]:
movies['tag']=movies['overview']+movies['genre_ids']+movies['cast']+movies['crew']
movies['tag']=movies['tag'].apply(lambda x:str(x)) #to resolve 'Float' object is not iterable error, we convert it into string
movies['tag']=movies['tag'].apply(lambda x:''.join([char for char in x if char not in string.punctuation])) #converting to string results in [ ' , " to be considered as well so removing these punctuations 
movies

C:\Users\Kashyap Ghimire\AppData\Local\Temp\ipykernel_14000\2391413916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tag']=movies['overview']+movies['genre_ids']+movies['cast']+movies['crew']
C:\Users\Kashyap Ghimire\AppData\Local\Temp\ipykernel_14000\2391413916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tag']=movies['tag'].apply(lambda x:str(x)) #to resolve 'Float' object is not iterable error, we convert it into string
C:\Users\Kashyap Ghimire\AppData\Local\Temp\ipykernel_14

,genre_ids,title,overview,cast,crew,id,tag
0,"[Drama, Crime]",The Godfather,"[Spanning, the, years, 1945, to, 1955, a, chro...",[Marlon Brando],[Francis Ford Coppola],238,Spanning the years 1945 to 1955 a chronicle of...
1,"[Drama, Crime]",The Shawshank Redemption,"[Framed, in, the, 1940s, for, the, double, mur...",[Tim Robbins],[Frank Darabont],278,Framed in the 1940s for the double murder of h...
2,"[Drama, Crime]",The Godfather Part II,"[In, the, continuing, saga, of, the, Corleone,...",[Al Pacino],[Francis Ford Coppola],240,In the continuing saga of the Corleone crime f...
3,"[Drama, History, War]",Schindler's List,"[The, true, story, of, how, businessman, Oskar...",[Liam Neeson],[Steven Spielberg],424,The true story of how businessman Oskar Schind...
4,"[Comedy, Drama, Romance]",Dilwale Dulhania Le Jayenge,"[Raj, is, a, rich, carefree, happygolucky, sec...",[Shah Rukh Khan],[Aditya Chopra],19404,Raj is a rich carefree happygolucky second gen...
...,...,...,...,...,...,...,...
9995,"[Horror, Thriller]",Dark Was the Night,"[An, evil, is, unleashed, in, a, small, town, ...",[Kevin Durand],[Jack Heller],300467,An evil is unleashed in a small town when a lo...
9996,"[Action, Crime, Thriller]",Machete Kills,"[ExFederale, agent, Machete, is, recruited, by...",[Danny Trejo],[Robert Rodriguez],106747,ExFederale agent Machete is recruited by the P...
9997,"[Adventure, Comedy, Family, Romance]",George of the Jungle,"[Deep, in, the, African, jungle, a, baby, name...",[Brendan Fraser],[Sam Weisman],10603,Deep in the African jungle a baby named George...
9998,"[Drama, Horror, Thriller]",Cadaver,"[In, the, aftermath, of, a, nuclear, disaster,...",[Gitte Witt],[Jarand Herdal],692969,In the aftermath of a nuclear disaster a starv...


In [15]:
movies=movies.drop(['overview','genre_ids','cast','crew'],axis=1)

In [16]:
#converting titles into lowercase:
movies['title_lcase']=movies['title'].apply(lambda x: x.lower())
movies

,title,id,tag,title_lcase
0,The Godfather,238,Spanning the years 1945 to 1955 a chronicle of...,the godfather
1,The Shawshank Redemption,278,Framed in the 1940s for the double murder of h...,the shawshank redemption
2,The Godfather Part II,240,In the continuing saga of the Corleone crime f...,the godfather part ii
3,Schindler's List,424,The true story of how businessman Oskar Schind...,schindler's list
4,Dilwale Dulhania Le Jayenge,19404,Raj is a rich carefree happygolucky second gen...,dilwale dulhania le jayenge
...,...,...,...,...
9995,Dark Was the Night,300467,An evil is unleashed in a small town when a lo...,dark was the night
9996,Machete Kills,106747,ExFederale agent Machete is recruited by the P...,machete kills
9997,George of the Jungle,10603,Deep in the African jungle a baby named George...,george of the jungle
9998,Cadaver,692969,In the aftermath of a nuclear disaster a starv...,cadaver


In [17]:
movies.isnull().sum()
# movies=movies.dropna()


title          0
id             0
tag            0
title_lcase    0
dtype: int64

In [18]:
movies

,title,id,tag,title_lcase
0,The Godfather,238,Spanning the years 1945 to 1955 a chronicle of...,the godfather
1,The Shawshank Redemption,278,Framed in the 1940s for the double murder of h...,the shawshank redemption
2,The Godfather Part II,240,In the continuing saga of the Corleone crime f...,the godfather part ii
3,Schindler's List,424,The true story of how businessman Oskar Schind...,schindler's list
4,Dilwale Dulhania Le Jayenge,19404,Raj is a rich carefree happygolucky second gen...,dilwale dulhania le jayenge
...,...,...,...,...
9995,Dark Was the Night,300467,An evil is unleashed in a small town when a lo...,dark was the night
9996,Machete Kills,106747,ExFederale agent Machete is recruited by the P...,machete kills
9997,George of the Jungle,10603,Deep in the African jungle a baby named George...,george of the jungle
9998,Cadaver,692969,In the aftermath of a nuclear disaster a starv...,cadaver


In [19]:
import nltk
from nltk.corpus import stopwords

# vectorization before calculating cosine similarity

In [20]:
cv=CountVectorizer(max_features=10000,stop_words='english')
# doc_vec = count_vectorizer.fit_transform(df1.iloc[0])

In [21]:
vector=cv.fit_transform(movies['tag']).toarray() 
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# calculating cosine similarity

In [22]:
similarity=cosine_similarity(vector)
similarity

array([[1.        , 0.11959122, 0.49580177, ..., 0.04721922, 0.09430419,
        0.03296902],
       [0.11959122, 1.        , 0.10390487, ..., 0.02597622, 0.06917145,
        0.03627381],
       [0.49580177, 0.10390487, 1.        , ..., 0.07692308, 0.10241831,
        0.03580574],
       ...,
       [0.04721922, 0.02597622, 0.07692308, ..., 1.        , 0.03413944,
        0.        ],
       [0.09430419, 0.06917145, 0.10241831, ..., 0.03413944, 1.        ,
        0.09534626],
       [0.03296902, 0.03627381, 0.03580574, ..., 0.        , 0.09534626,
        1.        ]])

In [23]:
def recommend(title):
    Index=movies[movies['title_lcase']==title.lower()].index[0]
    distance=sorted(list(enumerate(similarity[Index])),reverse=True, key=lambda vector:vector[1])
    for i in distance[1:6]:
        print(movies.iloc[i[0]].title)

In [24]:
recommend("Iron Man")

Iron Man 2
Iron Man 3
Avengers: Age of Ultron
G.O.R.A.
Insurgent


dumping into pickle file

In [25]:
import pickle
pickle.dump(movies,open('movies_data.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))